###### References: 
- Fluent Python, 2nd Edition, by Luciano Ramalho.

# Chapter 14: Inheritance: For Better or for Worse
## The Super() Function
## Subclassing Built-In Types
## Multiple Inheritance and Method Resolution Order

<img src="Ping pong.png" width="75%" />

In [1]:
class Root:  # <1>
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root):  # <2>
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root):  # <3>
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B):  # <4>
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

In [2]:
leaf1 = Leaf()

In [3]:
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [4]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


In [5]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

`diamond2.py`

In [6]:
class U():  # <2>
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()  # <3>

class LeafUA(U, A):  # <4>
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

In [7]:
u = U()

In [8]:
u.ping()

<__main__.U object at 0x10ba8d240>.ping() in U


AttributeError: 'super' object has no attribute 'ping'

In [9]:
leaf2 = LeafUA()

In [10]:
leaf2.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


In [11]:
LeafUA.__mro__

(__main__.LeafUA, __main__.U, __main__.A, __main__.Root, object)

In [12]:
class LeafAU(A, U):
    def ping(self):
        print(f'{self}.ping() in LeafAU')
        super().ping()

In [13]:
leaf2 = LeafAU()

In [14]:
leaf2.ping()

<instance of LeafAU>.ping() in LeafAU
<instance of LeafAU>.ping() in A
<instance of LeafAU>.ping() in Root


In [15]:
LeafAU.__mro__

(__main__.LeafAU, __main__.A, __main__.Root, __main__.U, object)

## Mixin Classes

Mixin class is designed to be subclassed together with at least one other class in a multiple inheritance arrangement.

### Case- Insensitive Mappings

In [16]:
import collections

In [17]:
def _upper(key):  # <1>
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:  # <2>
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

In [18]:
class UpperDict(UpperCaseMixin, collections.UserDict):  # <1>
    pass


In [19]:
class UpperCounter(UpperCaseMixin, collections.Counter):  # <2>
    """Specialized 'Counter' that uppercases string keys""" 

## Multiple Inheritance in the Real World

### Django

<img src="Django-view.png" width="75%" />
<img src="Django-views.png" width="75%" />                                  

# Chapter 18:  `with`, `match` and `else` Blocks

## Pattern Matching in `lis.py`: a Cases Study
### Scheme Syntax

    (define (mod m n))
        (-m (* n (quotient m n)))
        
    (define (gcd m n))
        (if (= n 0)
            m
            (gcd n (mod m n))))
            
    (display (gcd 18 45))

In [20]:
def mod(m, n):
    return m - (m // n * n)

def gcd(m, n):
    if n == 0:
        return m
    else:
        return gcd(n, mod(m,n))
    
print(gcd(18, 45))

9


Scheme syntax has no iterative control flow commands.

Iteration is done with recursion.

### Imports and Types

In [21]:
import math
import operator as op
from collections import ChainMap
from itertools import chain
from typing import Any, TypeAlias, NoReturn

Symbol: TypeAlias = str
Atom: TypeAlias = float | int | Symbol
Expression: TypeAlias = Atom | list

### The Parser

In [22]:
def parse(program: str) -> Expression:
    "Read a Scheme expression from a string."
    return read_from_tokens(tokenize(program))

def tokenize(s: str) -> list[str]:
    "Convert a string into a list of tokens."
    return s.replace('(', ' ( ').replace(')', ' ) ').split()

def read_from_tokens(tokens: list[str]) -> Expression:
    "Read an expression from a sequence of tokens."
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF while reading')
    token = tokens.pop(0)
    if '(' == token:
        exp = []
        while tokens[0] != ')':
            exp.append(read_from_tokens(tokens))
        tokens.pop(0)  # discard ')'
        return exp
    elif ')' == token:
        raise SyntaxError('unexpected )')
    else:
        return parse_atom(token)

In [23]:
from lis import parse

In [24]:
parse('1.5')

1.5

In [25]:
parse('ni!')

'ni!'

In [26]:
parse('(gcd 18 45)')

['gcd', 18, 45]

In [27]:
parse('''
    (define double
        (lambda (n)
            (* n 2)))''')

['define', 'double', ['lambda', ['n'], ['*', 'n', 2]]]

### The Environment

In [28]:
class Environment(ChainMap[Symbol, Any]):
    "A ChainMap that allows changing an item in-place."

    def change(self, key: Symbol, value: Any) -> None:
        "Find where key is defined and change the value there."
        for map in self.maps:
            if key in map:
                map[key] = value  # type: ignore[index]
                return
        raise KeyError(key)

In [29]:
from lis import Environment

In [30]:
inner_env = {'a': 2}

In [31]:
outer_env = {'a': 0, 'b':1}

In [32]:
env = Environment(inner_env, outer_env)

In [33]:
env['a']

2

In [34]:
env['a'] = 111
env['c'] = 222
env

Environment({'a': 111, 'c': 222}, {'a': 0, 'b': 1})

In [35]:
env.change('b', 333)
env

Environment({'a': 111, 'c': 222}, {'a': 0, 'b': 333})

In [36]:
def standard_env() -> Environment:
    "An environment with some Scheme standard procedures."
    env = Environment()
    env.update(vars(math))   # sin, cos, sqrt, pi, ...
    env.update({
            '+': op.add,
            '-': op.sub,
            '*': op.mul,
            '/': op.truediv,
            'quotient': op.floordiv,
            '>': op.gt,
            '<': op.lt,
            '>=': op.ge,
            '<=': op.le,
            '=': op.eq,
            'abs': abs,
            'append': lambda *args: list(chain(*args)),
            'apply': lambda proc, args: proc(*args),
            'begin': lambda *x: x[-1],
            'car': lambda x: x[0],
            'cdr': lambda x: x[1:],
            'cons': lambda x, y: [x] + y,
            'display': lambda x: print(lispstr(x)),
            'eq?': op.is_,
            'equal?': op.eq,
            'filter': lambda *args: list(filter(*args)),
            'length': len,
            'list': lambda *x: list(x),
            'list?': lambda x: isinstance(x, list),
            'map': lambda *args: list(map(*args)),
            'max': max,
            'min': min,
            'not': op.not_,
            'null?': lambda x: x == [],
            'number?': lambda x: isinstance(x, (int, float)),
            'procedure?': callable,
            'round': round,
            'symbol?': lambda x: isinstance(x, Symbol),
    })
    return 

## The REPL
(read-evel-print-loop)

In [37]:
def repl(prompt: str = 'lis.py> ') -> NoReturn:
    "A prompt-read-eval-print loop."
    global_env = Environment({}, standard_env())
    while True:
        ast = parse(input(prompt))
        val = evaluate(ast, global_env)
        if val is not None:
            print(lispstr(val))

In [38]:
def lispstr(exp: object) -> str:
    "Convert a Python object back into a Lisp-readable string."
    if isinstance(exp, list):
        return '(' + ' '.join(map(lispstr, exp)) + ')'
    else:
        return str(exp)

### The Evaluator

In [39]:
KEYWORDS = ['quote', 'if', 'lambda', 'define', 'set!']

In [40]:
def evaluate(exp: Expression, env: Environment) -> Any:
    "Evaluate an expression in an environment."
    match exp:
        case int(x) | float(x):
            return x
        case Symbol(var):
            return env[var]
        case ['quote', x]:
            return x
        case ['if', test, consequence, alternative]:
            if evaluate(test, env):
                return evaluate(consequence, env)
            else:
                return evaluate(alternative, env)
        case ['lambda', [*parms], *body] if body:
            return Procedure(parms, body, env)
        case ['define', Symbol(name), value_exp]:
            env[name] = evaluate(value_exp, env)
        case ['define', [Symbol(name), *parms], *body] if body:
            env[name] = Procedure(parms, body, env)
        case ['set!', Symbol(name), value_exp]:
            env.change(name, evaluate(value_exp, env))
        case [func_exp, *args] if func_exp not in KEYWORDS:
            proc = evaluate(func_exp, env)
            values = [evaluate(arg, env) for arg in args]
            return proc(*values)
        case _:
            raise SyntaxError(lispstr(exp))

### Procedure: A Class implementing a Closure

In [41]:
class Procedure:
    "A user-defined Scheme procedure."

    def __init__(  # called when a function is defined by the lambda or define
        self, parms: list[Symbol], body: list[Expression], env: Environment
    ):
        self.parms = parms  # <2>
        self.body = body
        self.env = env

    def __call__(self, *args: Expression) -> Any:  # <3>
        local_env = dict(zip(self.parms, args))  # <4>
        env = Environment(local_env, self.env)  # <5>
        for exp in self.body:  # <6>
            result = evaluate(exp, env)
        return result  # <7>

### Using OR-patterns

    case int(x) | float(x):
        return x
        
    
    # (λ (a b) (/ (+ a b) 2))
    case ['lambda' | 'λ', [*parms], *body] if body:
        return Procudure(parms, body, env)